<a href="https://colab.research.google.com/github/UdayalChaudhary/Task-Audio-to-text/blob/main/app_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q streamlit openai-whisper torch torchaudio torchvision
!pip install -q moviepy ffmpeg-python pydub
!pip install -q python-docx reportlab numpy pandas
!pip install -q pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 16.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 108.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 128.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 65.8 MB/s eta 0:00:00


In [ ]:
!apt-get update
!apt-get install -y ffmpeg

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,233 kB]
Get:5 https://cli.github.com/packages stable/main amd64 Packages [345 B]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,864 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu

In [ ]:
!pip install -q transformers sentencepiece

In [ ]:
!pip install librosa plotly

In [ ]:
%%writefile app.py
import os
import json
import subprocess
import torch
import streamlit as st
import whisper
from moviepy.editor import VideoFileClip
import streamlit.components.v1 as components
import base64
import re
import librosa
import numpy as np
import plotly.graph_objects as go
from io import BytesIO
from docx import Document
from docx.shared import RGBColor
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas

# ----------------- PYTORCH STABILITY FIX -----------------
torch.backends.cuda.enable_flash_sdp(False)
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_math_sdp(True)

# ----------------- STREAMLIT CONFIG -----------------
st.set_page_config(layout="wide")
st.title("Offline Audio / Video Intelligence Dashboard")

TEMP_DIR = "temp"
os.makedirs(TEMP_DIR, exist_ok=True)

# ----------------- THREAT WORD LIST -----------------
THREAT_WORDS = {
    "bomb","explosive","grenade","rifle","pistol","gun","weapon",
    "ammo","ammunition","firearm","detonator","detonate","trigger",
    "attack","assault","kill","shoot","stab","execute","murder",
    "terror","terrorist","hostage","kidnap","abduct",
    "gathering","assemble","target","plan","coordinate",
    "chemicals","materials","supplies","components","fuel",
    "bridge","airport","station","crowd","building",
    "hack","breach","sabotage","arm"
}

# ----------------- LOAD WHISPER -----------------
@st.cache_resource
def load_whisper():
    return whisper.load_model("small").to("cuda")

whisper_model = load_whisper()

# ----------------- FILE UPLOAD -----------------
uploaded_file = st.file_uploader(
    "Upload Video or Audio",
    type=["mp4", "mkv", "mov", "wav", "mp3"]
)

# ----------------- HELPERS -----------------
def format_time(seconds):
    m = int(seconds // 60)
    s = int(seconds % 60)
    return f"{m:02d}:{s:02d}"

def highlight_threat_words(text):
    def repl(match):
        w = match.group(0)
        if w.lower() in THREAT_WORDS:
            return f"<span class='risk'>{w}</span>"
        return w
    return re.sub(r"\b\w+\b", repl, text)

def strip_html(html):
    return re.sub(r"<.*?>", "", html)

def split_with_threats(text):
    tokens = re.findall(r"\w+|\W+", text)
    return [("threat", t) if t.strip().lower() in THREAT_WORDS else ("normal", t) for t in tokens]

# ----------------- PITCH -----------------
def extract_pitch(audio_path):
    y, sr = librosa.load(audio_path, sr=16000)
    pitches, mags = librosa.piptrack(y=y, sr=sr)
    vals, times = [], []
    for t in range(pitches.shape[1]):
        i = mags[:, t].argmax()
        p = pitches[i, t]
        if p > 0:
            vals.append(p)
            times.append(t * len(y) / sr / pitches.shape[1])
    return np.array(times), np.array(vals)

def pitch_dashboard(times, pitches):
    if len(pitches) == 0:
        st.warning("No pitch detected.")
        return
    norm = (pitches - pitches.min()) / (pitches.max() - pitches.min() + 1e-6)
    colors = ["green" if p < .33 else "orange" if p < .66 else "red" for p in norm]
    fig = go.Figure(go.Scatter(x=times, y=pitches, mode="markers",
                               marker=dict(color=colors, size=4)))
    fig.update_layout(title="🎵 Pitch Modulation (Emotion Indicator)",
                      xaxis_title="Time (seconds)",
                      yaxis_title="Pitch (Hz)",
                      height=300)
    st.plotly_chart(fig, use_container_width=True)

# ----------------- DOWNLOAD BUILDERS -----------------
def create_txt(segments):
    out = []
    for s in segments:
        text = strip_html(s["html"])
        parts = split_with_threats(text)
        line = ""
        for k, w in parts:
            line += f"__{w}__" if k == "threat" else w
        out.append(f"[{s['start_ts']} → {s['end_ts']}] {line}")
    return "\n\n".join(out).encode("utf-8")

def create_docx(segments):
    doc = Document()
    for s in segments:
        p = doc.add_paragraph()
        p.add_run(f"[{s['start_ts']} → {s['end_ts']}] ").italic = True
        for k, w in split_with_threats(strip_html(s["html"])):
            r = p.add_run(w)
            if k == "threat":
                r.bold = True
                r.underline = True
                r.font.color.rgb = RGBColor(176, 0, 32)
    buf = BytesIO()
    doc.save(buf)
    buf.seek(0)
    return buf

def create_pdf(segments):
    buf = BytesIO()
    c = canvas.Canvas(buf, pagesize=A4)
    w, h = A4
    y = h - 40
    for s in segments:
        c.setFont("Helvetica-Oblique", 9)
        c.drawString(40, y, f"[{s['start_ts']} → {s['end_ts']}]")
        y -= 14
        x = 40
        for k, word in split_with_threats(strip_html(s["html"])):
            if y < 40:
                c.showPage()
                y = h - 40
                x = 40
            if k == "threat":
                c.setFont("Helvetica-Bold", 10)
                c.setFillColorRGB(.69, 0, 0)
                c.drawString(x, y, word)
                c.line(x, y-2, x+c.stringWidth(word, "Helvetica-Bold", 10), y-2)
            else:
                c.setFont("Helvetica", 10)
                c.setFillColorRGB(0, 0, 0)
                c.drawString(x, y, word)
            x += c.stringWidth(word, c._fontname, 10)
        y -= 18
    c.save()
    buf.seek(0)
    return buf

# ----------------- HTML PLAYER -----------------
def render_synced_player(media_path, segments, is_video=True):
    with open(media_path, "rb") as f:
        media_b64 = base64.b64encode(f.read()).decode()

    mime = "video/mp4" if is_video else "audio/wav"

    html = f"""
    <html>
    <head>
        <style>
            body {{ font-family: Segoe UI, Arial; }}
            .container {{
                display: grid;
                grid-template-columns: 58% 42%;
                gap: 24px;
            }}
            .player {{
                position: relative;
                background: black;
                border-radius: 14px;
                overflow: hidden;
                box-shadow: 0 6px 18px rgba(0,0,0,.15);
            }}
            video, audio {{ width: 100%; }}
            .caption {{
                position: absolute;
                bottom: 18px;
                left: 50%;
                transform: translateX(-50%);
                width: 92%;
                background: rgba(0,0,0,.75);
                color: white;
                padding: 14px;
                font-size: 20px;
                text-align: center;
                border-radius: 10px;
            }}
            .transcript {{
                background: #f9fafb;
                border-radius: 14px;
                padding: 16px;
                height: 520px;
                overflow-y: auto;
                box-shadow: inset 0 0 0 1px #e5e7eb;
            }}
            .seg {{
                padding: 10px;
                margin-bottom: 10px;
                border-radius: 10px;
                cursor: pointer;
            }}
            .seg:hover {{ background: #eef2ff; }}
            .active {{
                background: #e0edff;
                box-shadow: inset 4px 0 0 #3b82f6;
            }}
            .timestamp {{
                font-size: 12px;
                color: #6b7280;
                display: block;
                margin-bottom: 4px;
            }}
            .risk {{
                color: #b00020;
                font-weight: 700;
                text-decoration: underline;
            }}
        </style>
    </head>

    <body>
        <div class="container">
            <div class="player">
                {"<video id='playerEl' controls>" if is_video else "<audio id='playerEl' controls>"}
                    <source src="data:{mime};base64,{media_b64}">
                {"</video>" if is_video else "</audio>"}
                <div id="caption" class="caption"></div>
            </div>
            <div class="transcript" id="transcript"></div>
        </div>

        <script>
            const segments = {json.dumps(segments)};
            const transcript = document.getElementById("transcript");
            const player = document.getElementById("playerEl");
            const caption = document.getElementById("caption");

            segments.forEach(seg => {{
                const div = document.createElement("div");
                div.className = "seg";
                div.dataset.start = seg.start;
                div.dataset.end = seg.end;
                div.innerHTML = `
                    <span class="timestamp">[${{seg.start_ts}} → ${{seg.end_ts}}]</span>
                    ${{seg.html}}
                `;
                div.onclick = () => {{
                    player.currentTime = seg.start;
                    player.play();
                }};
                transcript.appendChild(div);
            }});

            player.addEventListener("timeupdate", () => {{
                const t = player.currentTime;
                let activeHTML = "";
                document.querySelectorAll(".seg").forEach(el => {{
                    const s = parseFloat(el.dataset.start);
                    const e = parseFloat(el.dataset.end);
                    const active = t >= s && t <= e;
                    el.classList.toggle("active", active);
                    if (active) activeHTML = el.innerHTML;
                }});
                caption.innerHTML = activeHTML;
            }});
        </script>
    </body>
    </html>
    """

    components.html(html, height=660, scrolling=True)
# ----------------- MAIN -----------------
if uploaded_file:
    path = os.path.join(TEMP_DIR, uploaded_file.name)
    with open(path, "wb") as f: f.write(uploaded_file.read())
    audio = os.path.join(TEMP_DIR, "audio.wav")

    if uploaded_file.name.endswith(("mp4","mkv","mov")):
        VideoFileClip(path).audio.write_audiofile(audio, logger=None)
        media, is_video = path, True
    else:
        subprocess.run(["ffmpeg","-y","-i",path,"-ac","1","-ar","16000",audio])
        media, is_video = audio, False

    subprocess.run(["ffmpeg","-y","-i",audio,"-ac","1","-ar","16000",audio])

    with st.spinner("Transcribing..."):
        r = whisper_model.transcribe(audio, verbose=False)

    segments=[]
    for s in r["segments"]:
        segments.append({
            "start": s["start"],
            "end": s["end"],
            "start_ts": format_time(s["start"]),
            "end_ts": format_time(s["end"]),
            "html": highlight_threat_words(s["text"])
        })

    with st.expander("🎵 Pitch & Emotion Analysis", expanded=True):
        t,p = extract_pitch(audio)
        pitch_dashboard(t,p)

    render_synced_player(media, segments, is_video)

    st.markdown("Download Transcript")
    c1,c2,c3 = st.columns(3)
    c1.download_button("⬇ TXT", create_txt(segments), "transcript.txt")
    c2.download_button("⬇ Word", create_docx(segments), "transcript.docx")
    c3.download_button("⬇ PDF", create_pdf(segments), "transcript.pdf")


Overwriting app.py


In [ ]:
!ls

app.py	sample_data


In [ ]:
!pip install -q pyngrok

In [ ]:
from pyngrok import ngrok

ngrok.set_auth_token("3823bMlhsHQEAMJ6Npt41ASTS3Q_3MSHFYxgKrJY7a34KDZoz")

In [ ]:
!streamlit run app.py --server.port 8501 --server.address 0.0.0.0




  You can now view your Streamlit app in your browser.

  URL: http://0.0.0.0:8501

error: XDG_RUNTIME_DIR not set in the environment.
ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5701:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM default
ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c

In [ ]:
from pyngrok import ngrok

ngrok.kill()
public_url = ngrok.connect(8501)
print(public_url)

NgrokTunnel: "https://fountained-expansively-florencio.ngrok-free.dev" -> "http://localhost:8501"
